# Treinamento Random Forest - Baseline UTA-RLDD

Este notebook implementa um modelo baseline usando Random Forest para detecção de fadiga no dataset UTA-RLDD.

**Pré-requisito**: Execute o notebook de preparação de dados primeiro.

**Meta**: Atingir 75-85% de acurácia como baseline

---

## 1. Importações

In [ ]:
# Bibliotecas principais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
import pickle
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score, 
    roc_auc_score
)

# Análise estatística
from scipy import stats

print("Bibliotecas carregadas com sucesso!")
print(f"Diretório atual: {Path.cwd()}")

## 2. Configuração

In [ ]:
# Caminhos dos dados
DIRETORIO_DADOS = Path("processed_data_uta_rldd_CORRECTED")
DIRETORIO_MODELOS = DIRETORIO_DADOS / "rf_models"
DIRETORIO_MODELOS.mkdir(exist_ok=True)

# Nomes das características originais
NOMES_SINAIS = ['PERCLOS', 'MAR', 'BLINK_RATE', 'HEAD_STABILITY']
NOMES_STATS = ['mean', 'std', 'median', 'min', 'max', 'range', 'q25', 'q75', 'trend', 'zcr', 'autocorr']

# Criar nomes das features
NOMES_FEATURES = []
for sinal in NOMES_SINAIS:
    for stat in NOMES_STATS:
        NOMES_FEATURES.append(f"{sinal}_{stat}")

print(f"Total de features a extrair: {len(NOMES_FEATURES)}")
print(f"Diretório dos dados: {DIRETORIO_DADOS}")
print(f"Diretório dos modelos: {DIRETORIO_MODELOS}")

## 3. Carregamento dos Dados

In [ ]:
def carregar_dados_processados():
    """Carrega sequências e labels do notebook de preparação de dados"""
    
    try:
        # Tentar carregar arquivos numpy
        arquivo_X = DIRETORIO_DADOS / "X_sequences.npy"
        arquivo_y = DIRETORIO_DADOS / "y_labels.npy"
        
        if arquivo_X.exists() and arquivo_y.exists():
            X_sequences = np.load(arquivo_X)
            y_labels = np.load(arquivo_y)
            print(f"Dados carregados de {arquivo_X} e {arquivo_y}")
            return {'X_sequences': X_sequences, 'y_labels': y_labels}
        
        # Listar arquivos disponíveis
        arquivos_disponiveis = list(DIRETORIO_DADOS.glob("*.pkl")) + list(DIRETORIO_DADOS.glob("*.npy"))
        print(f"Arquivos disponíveis em {DIRETORIO_DADOS}:")
        for arquivo in arquivos_disponiveis:
            print(f"  - {arquivo.name}")
        
        raise FileNotFoundError("Não foi possível encontrar os dados processados")
        
    except Exception as e:
        print(f"Erro ao carregar dados: {e}")
        print("Execute o notebook de preparação de dados primeiro.")
        raise

# Carregar os dados
dados = carregar_dados_processados()

X_sequences = dados['X_sequences']
y_labels = dados['y_labels']

print(f"Formato das sequências: {X_sequences.shape}")
print(f"Formato dos labels: {y_labels.shape}")
print(f"Labels únicos: {np.unique(y_labels)}")
print(f"Distribuição dos labels: {np.bincount(y_labels)}")

## 4. Extração de Características Estatísticas

Converte sequências temporais (90 frames × 4 features) em características estatísticas para ML clássico.

In [ ]:
class ExtratorCaracteristicasFadiga:
    """Extrai características estatísticas de sequências temporais para ML clássico"""
    
    def __init__(self):
        self.nomes_features = NOMES_FEATURES.copy()
    
    def inclinacao_tendencia(self, sinal):
        """Calcula inclinação da tendência linear"""
        if len(sinal) < 2:
            return 0.0
        x = np.arange(len(sinal))
        try:
            inclinacao, _, _, _, _ = stats.linregress(x, sinal)
            return inclinacao if not np.isnan(inclinacao) else 0.0
        except:
            return 0.0
    
    def taxa_cruzamento_zero(self, sinal):
        """Taxa de cruzamento do zero (medida de variabilidade)"""
        if len(sinal) < 2:
            return 0.0
        centrado_na_media = sinal - np.mean(sinal)
        cruzamentos = np.sum(np.diff(np.sign(centrado_na_media)) != 0)
        return cruzamentos / len(sinal)
    
    def autocorr_lag1(self, sinal):
        """Autocorrelação de lag-1 (consistência temporal)"""
        if len(sinal) < 3:
            return 0.0
        try:
            corr = np.corrcoef(sinal[:-1], sinal[1:])[0, 1]
            return corr if not np.isnan(corr) else 0.0
        except:
            return 0.0
    
    def extrair_features_sinal(self, sinal):
        """Extrai 11 características estatísticas de um único sinal"""
        features = [
            np.mean(sinal),                    # Tendência central
            np.std(sinal),                     # Variabilidade  
            np.median(sinal),                  # Tendência central robusta
            np.min(sinal),                     # Medidas de amplitude
            np.max(sinal),
            np.ptp(sinal),                     # Peak-to-peak (max - min)
            np.percentile(sinal, 25),          # Quartis da distribuição
            np.percentile(sinal, 75),
            self.inclinacao_tendencia(sinal), # Dinâmica temporal
            self.taxa_cruzamento_zero(sinal),
            self.autocorr_lag1(sinal)          # Correlação temporal
        ]
        return features
    
    def transform(self, X_sequences):
        """
        Transforma sequências temporais em características estatísticas
        
        Input: (n_samples, 90, 4) - sequências temporais
        Output: (n_samples, 44) - características estatísticas
        """
        n_samples = X_sequences.shape[0]
        n_features = len(self.nomes_features)
        
        X_features = np.zeros((n_samples, n_features))
        
        print(f"Extraindo características para {n_samples} sequências...")
        
        for i in range(n_samples):
            if i % 1000 == 0:
                print(f"  Processando amostra {i}/{n_samples}")
            
            sequencia = X_sequences[i]  # Shape: (90, 4)
            features_amostra = []
            
            # Extrair features para cada um dos 4 sinais
            for idx_sinal in range(4):  # PERCLOS, MAR, BLINK_RATE, HEAD_STABILITY
                sinal = sequencia[:, idx_sinal]
                features_sinal = self.extrair_features_sinal(sinal)
                features_amostra.extend(features_sinal)
            
            X_features[i] = features_amostra
        
        print(f"Extração de características concluída! Shape: {X_features.shape}")
        return X_features

# Inicializar extrator
extrator_features = ExtratorCaracteristicasFadiga()
print(f"Extrator inicializado com {len(extrator_features.nomes_features)} características")

## 5. Extrair Características das Sequências

In [ ]:
# Extrair características estatísticas das sequências temporais
print("Extraindo características estatísticas das sequências temporais...")
print(f"Formato de entrada: {X_sequences.shape} (amostras, passos_temporais, características)")

X_features = extrator_features.transform(X_sequences)

print(f"\nResultados da extração:")
print(f"  Sequências originais: {X_sequences.shape}")
print(f"  Características extraídas: {X_features.shape}")
print(f"  Features por sinal: 11")
print(f"  Total de sinais: 4 (PERCLOS, MAR, BLINK_RATE, HEAD_STABILITY)")

# Verificar qualidade dos dados
nan_count = np.isnan(X_features).sum()
inf_count = np.isinf(X_features).sum()
print(f"\nVerificação de qualidade:")
print(f"  Valores NaN: {nan_count}")
print(f"  Valores infinitos: {inf_count}")

if nan_count > 0 or inf_count > 0:
    print("  Corrigindo valores NaN/Inf...")
    X_features = np.nan_to_num(X_features, nan=0.0, posinf=0.0, neginf=0.0)

print("\nEstatísticas das características:")
print(f"  Média: {np.mean(X_features):.4f}")
print(f"  Desvio padrão: {np.std(X_features):.4f}")
print(f"  Min: {np.min(X_features):.4f}")
print(f"  Max: {np.max(X_features):.4f}")

## 6. Divisão e Pré-processamento dos Dados

In [ ]:
# Dividir dados mantendo balanceamento das classes
print("Dividindo dados em conjuntos de treino/teste...")
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y_labels, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_labels
)

print(f"Conjunto de treino: {X_train.shape[0]} amostras")
print(f"Conjunto de teste: {X_test.shape[0]} amostras")
print(f"Distribuição treino: {np.bincount(y_train)}")
print(f"Distribuição teste: {np.bincount(y_test)}")

# Normalização das características
print("\nAplicando normalização...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Dados de treino normalizados: {X_train_scaled.shape}")
print(f"Dados de teste normalizados: {X_test_scaled.shape}")

# Verificar normalização
print(f"\nEstatísticas após normalização (treino):")
print(f"  Média: {np.mean(X_train_scaled):.6f}")
print(f"  Desvio padrão: {np.std(X_train_scaled):.6f}")

## 7. Treinamento do Modelo Random Forest

In [ ]:
# Inicializar Random Forest baseline
print("Treinando classificador Random Forest baseline...")

rf_baseline = RandomForestClassifier(
    n_estimators=100,           # 100 árvores
    max_depth=10,               # Evitar overfitting
    min_samples_split=5,        # Evitar overfitting
    min_samples_leaf=2,         # Evitar overfitting
    class_weight='balanced',    # Lidar com desbalanceamento
    random_state=42,            # Reprodutibilidade
    n_jobs=-1                   # Usar todos os cores
)

# Treinar o modelo
rf_baseline.fit(X_train_scaled, y_train)

print("Treinamento Random Forest concluído!")
print(f"Número de características usadas: {rf_baseline.n_features_in_}")
print(f"Número de árvores: {rf_baseline.n_estimators}")
print(f"Classes: {rf_baseline.classes_}")

## 8. Avaliação do Modelo

In [ ]:
# Fazer predições
print("Avaliando modelo Random Forest baseline...")

y_train_pred = rf_baseline.predict(X_train_scaled)
y_test_pred = rf_baseline.predict(X_test_scaled)
y_test_proba = rf_baseline.predict_proba(X_test_scaled)

# Calcular métricas
acuracia_treino = accuracy_score(y_train, y_train_pred)
acuracia_teste = accuracy_score(y_test, y_test_pred)

# Determinar nomes das classes
classes_unicas = np.unique(y_test)
print(f"Classes detectadas: {classes_unicas}")

# Mapear labels para nomes
mapeamento_classes = {0: 'Alert', 5: 'Low_Vigilant', 10: 'Drowsy'}
nomes_classes = [mapeamento_classes.get(cls, f'Classe_{cls}') for cls in sorted(classes_unicas)]

print(f"\n=== RESULTADOS BASELINE ===")
print(f"Acurácia Treino: {acuracia_treino:.4f} ({acuracia_treino*100:.2f}%)")
print(f"Acurácia Teste: {acuracia_teste:.4f} ({acuracia_teste*100:.2f}%)")
print(f"Tipo do problema: {len(classes_unicas)}-classe")
print(f"Classes: {nomes_classes}")

# Relatório detalhado
print(f"\n=== RELATÓRIO DETALHADO ===")
print(classification_report(y_test, y_test_pred, target_names=nomes_classes))

# Matriz de confusão
print(f"\n=== MATRIZ DE CONFUSÃO ===")
cm = confusion_matrix(y_test, y_test_pred)
print(cm)
print(f"Linhas: Labels verdadeiros, Colunas: Predições")
print(f"Ordem das classes: {nomes_classes}")

# ROC AUC
try:
    if len(classes_unicas) == 2:
        roc_auc = roc_auc_score(y_test, y_test_proba[:, 1])
        print(f"\nROC AUC Score: {roc_auc:.4f}")
    else:
        roc_auc = roc_auc_score(y_test, y_test_proba, multi_class='ovr', average='macro')
        print(f"\nROC AUC Score (macro-avg): {roc_auc:.4f}")
except Exception as e:
    print(f"\nROC AUC: Não foi possível calcular - {str(e)}")

# Avaliação de performance
print(f"\n=== AVALIAÇÃO DE PERFORMANCE ===")
if acuracia_teste >= 0.85:
    print("🟢 EXCELENTE: Performance de nível clínico (≥85%)")
elif acuracia_teste >= 0.80:
    print("🟡 MUITO BOM: Performance de alta qualidade (≥80%)")
elif acuracia_teste >= 0.75:
    print("🔵 BOM: Performance aceitável (≥75%)")
elif acuracia_teste >= 0.70:
    print("🟡 RAZOÁVEL: Performance mínima aceitável (≥70%)")
else:
    print("🔴 RUIM: Abaixo da performance aceitável (<70%)")

print(f"Meta: 75-85% de acurácia para uso clínico")

## 9. Análise de Importância das Características

In [ ]:
# Análise de importância das características
print("Analisando importância das características...")

# Obter importância das características do Random Forest
scores_importancia = rf_baseline.feature_importances_
df_importancia = pd.DataFrame({
    'caracteristica': extrator_features.nomes_features,
    'importancia': scores_importancia
}).sort_values('importancia', ascending=False)

# Mostrar top 20 características mais importantes
print(f"\n=== TOP 20 CARACTERÍSTICAS MAIS IMPORTANTES ===")
print(df_importancia.head(20).to_string(index=False))

# Agrupar por tipo de sinal
importancia_por_sinal = {}
for sinal in NOMES_SINAIS:
    features_sinal = df_importancia[df_importancia['caracteristica'].str.startswith(sinal)]
    importancia_por_sinal[sinal] = features_sinal['importancia'].sum()

print(f"\n=== IMPORTÂNCIA POR TIPO DE SINAL ===")
for sinal, importancia in sorted(importancia_por_sinal.items(), key=lambda x: x[1], reverse=True):
    print(f"{sinal:15} {importancia:.4f} ({importancia*100:.2f}%)")

# Visualizar importância das características
plt.figure(figsize=(12, 8))

# Top 15 características
top_features = df_importancia.head(15)
plt.subplot(2, 1, 1)
plt.barh(range(len(top_features)), top_features['importancia'])
plt.yticks(range(len(top_features)), top_features['caracteristica'])
plt.xlabel('Importância da Característica')
plt.title('Top 15 Características Mais Importantes')
plt.gca().invert_yaxis()

# Importância por sinal
plt.subplot(2, 1, 2)
sinais = list(importancia_por_sinal.keys())
importancias = list(importancia_por_sinal.values())
plt.bar(sinais, importancias)
plt.xlabel('Tipo de Sinal')
plt.ylabel('Importância Total')
plt.title('Importância das Características por Tipo de Sinal')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Salvar importância das características
df_importancia.to_csv(DIRETORIO_MODELOS / 'importancia_caracteristicas.csv', index=False)
print(f"\nImportância das características salva em: {DIRETORIO_MODELOS / 'importancia_caracteristicas.csv'}")

## 10. Validação Cruzada Temporal

In [ ]:
# Validação cruzada temporal para avaliar estabilidade
print("Executando validação cruzada temporal...")

# Usar TimeSeriesSplit para respeitar ordem temporal
tscv = TimeSeriesSplit(n_splits=5)

# Scores de validação cruzada
cv_scores = cross_val_score(
    rf_baseline, X_train_scaled, y_train, 
    cv=tscv, scoring='accuracy', n_jobs=-1
)

print(f"\n=== RESULTADOS DA VALIDAÇÃO CRUZADA TEMPORAL ===")
print(f"Scores individuais: {cv_scores}")
print(f"Score médio: {cv_scores.mean():.4f} ({cv_scores.mean()*100:.2f}%)")
print(f"Desvio padrão: {cv_scores.std():.4f} (±{cv_scores.std()*100:.2f}%)")
print(f"Intervalo de confiança 95%: [{cv_scores.mean() - 2*cv_scores.std():.4f}, {cv_scores.mean() + 2*cv_scores.std():.4f}]")

# Avaliação de estabilidade
if cv_scores.std() < 0.05:
    print("\n🟢 ESTÁVEL: Baixa variância entre folds (<5%)")
elif cv_scores.std() < 0.10:
    print("\n🟡 MODERADO: Variância aceitável entre folds (<10%)")
else:
    print("\n🔴 INSTÁVEL: Alta variância entre folds (≥10%)")

# Plotar resultados da validação cruzada
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(cv_scores) + 1), cv_scores, 'bo-', linewidth=2, markersize=8)
plt.axhline(y=cv_scores.mean(), color='r', linestyle='--', label=f'Média: {cv_scores.mean():.3f}')
plt.fill_between(range(1, len(cv_scores) + 1), 
                 cv_scores.mean() - cv_scores.std(), 
                 cv_scores.mean() + cv_scores.std(), 
                 alpha=0.2, color='red')
plt.xlabel('Fold')
plt.ylabel('Acurácia')
plt.title('Scores da Validação Cruzada')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(cv_scores, bins=5, alpha=0.7, edgecolor='black')
plt.axvline(cv_scores.mean(), color='red', linestyle='--', linewidth=2, label=f'Média: {cv_scores.mean():.3f}')
plt.xlabel('Acurácia')
plt.ylabel('Frequência')
plt.title('Distribuição dos Scores CV')
plt.legend()

plt.tight_layout()
plt.show()

## 11. Salvar Modelo Treinado

In [ ]:
# Salvar modelos treinados e componentes de pré-processamento
print("Salvando modelos treinados e componentes...")

# Salvar modelo Random Forest baseline
caminho_modelo_rf = DIRETORIO_MODELOS / 'rf_baseline_model.joblib'
joblib.dump(rf_baseline, caminho_modelo_rf)
print(f"Random Forest baseline salvo em: {caminho_modelo_rf}")

# Salvar o scaler
caminho_scaler = DIRETORIO_MODELOS / 'feature_scaler.joblib'
joblib.dump(scaler, caminho_scaler)
print(f"Scaler salvo em: {caminho_scaler}")

# Salvar extrator de características
caminho_extrator = DIRETORIO_MODELOS / 'feature_extractor.joblib'
joblib.dump(extrator_features, caminho_extrator)
print(f"Extrator de características salvo em: {caminho_extrator}")

# Salvar informações das classes
info_classes = {
    'classes': rf_baseline.classes_.tolist(),
    'n_classes': len(rf_baseline.classes_),
    'mapeamento_classes': {0: 'Alert', 5: 'Low_Vigilant', 10: 'Drowsy'}
}
import json
caminho_info_classes = DIRETORIO_MODELOS / 'class_info.json'
with open(caminho_info_classes, 'w') as f:
    json.dump(info_classes, f, indent=2)
print(f"Informações das classes salvas em: {caminho_info_classes}")

# Código para pipeline de deploy
codigo_deploy = '''import joblib
import numpy as np
import json
from pathlib import Path

class PipelinePredicaoFadiga:
    """Pipeline completo para predição de fadiga usando Random Forest"""
    
    def __init__(self, diretorio_modelo):
        diretorio_modelo = Path(diretorio_modelo)
        
        # Carregar componentes
        self.extrator_features = joblib.load(diretorio_modelo / 'feature_extractor.joblib')
        self.scaler = joblib.load(diretorio_modelo / 'feature_scaler.joblib')
        self.classificador = joblib.load(diretorio_modelo / 'rf_baseline_model.joblib')
        
        # Carregar informações das classes
        with open(diretorio_modelo / 'class_info.json', 'r') as f:
            self.info_classes = json.load(f)
        
        self.classes = self.info_classes['classes']
        self.mapeamento_classes = self.info_classes['mapeamento_classes']
    
    def predizer_sequencia(self, sequencia):
        """Prediz fadiga de uma única sequência de 90 frames"""
        # Adicionar dimensão de batch
        sequencias = np.expand_dims(sequencia, axis=0)
        
        # Extrair características
        features = self.extrator_features.transform(sequencias)
        
        # Normalizar características
        features_normalizadas = self.scaler.transform(features)
        
        # Fazer predição
        predicao = self.classificador.predict(features_normalizadas)[0]
        probabilidades_array = self.classificador.predict_proba(features_normalizadas)[0]
        
        # Criar dicionário de probabilidades
        probabilidades = {}
        for i, label_classe in enumerate(self.classes):
            nome_classe = self.mapeamento_classes.get(str(label_classe), f'Classe_{label_classe}')
            probabilidades[nome_classe] = float(probabilidades_array[i])
        
        # Obter nome da classe predita
        nome_classe_predita = self.mapeamento_classes.get(str(predicao), f'Classe_{predicao}')
        
        return predicao, probabilidades, nome_classe_predita

# Exemplo de uso:
# pipeline = PipelinePredicaoFadiga('processed_data_uta_rldd_CORRECTED/rf_models')
# predicao, probabilidades, nome_classe = pipeline.predizer_sequencia(dados_sequencia)
# print(f"Predição: {nome_classe} (label: {predicao})")
# print(f"Probabilidades: {probabilidades}")
'''

# Salvar código de deploy
caminho_deploy = DIRETORIO_MODELOS / 'pipeline_deploy.py'
with open(caminho_deploy, 'w') as f:
    f.write(codigo_deploy)
print(f"Pipeline de deploy salvo em: {caminho_deploy}")

print(f"\n=== TREINAMENTO CONCLUÍDO COM SUCESSO ===")
print(f"✅ Modelo Random Forest treinado e salvo")
print(f"✅ Pipeline de extração de características salvo")
print(f"✅ Componentes de pré-processamento salvos")
print(f"✅ Informações das classes salvas")
print(f"✅ Código de deploy gerado")
print(f"✅ Performance do modelo: {acuracia_teste:.1%}")
print(f"\nLocalização dos arquivos: {DIRETORIO_MODELOS}")
print(f"\nModelo suporta classificação de {len(rf_baseline.classes_)} classes:")
for cls in rf_baseline.classes_:
    nome_classe = mapeamento_classes.get(cls, f'Classe_{cls}')
    print(f"  - Classe {cls}: {nome_classe}")